In [1]:
# read it in to inspect it
with open('cleananime.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3726522


In [3]:
# let's look at the first 1000 characters
print(text[:200000])

 Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
 Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
 Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
 A Day Before Us is an animated Korean drama of a special and sweet love story of the following four main characters; Yeo Reum, Ha Eun, Kim Wook, and Yeon Woo.
 "Ken always disguised herself as a man. She was totally confused about why she got entangled with that man. Prince Milton

 After being cheated on by her boyfriend, Estrella, as a rookie, joined a video game club and accidentally ran into five wealthy young players and a video game t

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^`abcdefghijklmnopqrstuvwxyz|~ ­´о​‘’‚“”
99


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[68, 69, 69, 1, 80, 68, 65, 78, 65]
hii there


In [6]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([3726522]) torch.int64
tensor([ 1, 49, 75, 65, 78, 81,  1, 80, 61, 71, 65, 79,  1, 61,  1, 80, 65, 79,
        80,  1, 79, 75,  1, 79, 68, 65,  1, 63, 61, 74,  1, 65, 74, 80, 65, 78,
         1, 80, 68, 65,  1, 79, 61, 73, 65,  1, 68, 69, 67, 68,  1, 79, 63, 68,
        75, 75, 72,  1, 61, 79,  1, 47, 81, 74, 13,  1, 80, 68, 65,  1, 67, 69,
        78, 72,  1, 79, 68, 65,  1, 72, 69, 71, 65, 79, 15,  1, 48, 68, 65,  1,
        76, 61, 79, 79, 65, 79, 13,  1, 62, 81, 80,  1, 83, 68, 65, 74,  1, 79,
        68, 65,  1, 67, 75, 65, 79, 89, 89,  1, 80, 75,  1, 80, 65, 72, 72,  1,
        47, 81, 74, 13,  1, 79, 68, 65,  1, 66, 69, 74, 64, 79,  1, 68, 65, 78,
         1, 68, 81, 67, 67, 69, 74, 67,  1, 61,  1, 67, 69, 78, 72,  1, 79, 68,
        65, 95, 79,  1, 74, 65, 82, 65, 78,  1, 79, 65, 65, 74,  1, 62, 65, 66,
        75, 78, 65, 15,  0,  1, 49, 75, 65, 78, 81,  1, 80, 61, 71, 65, 79,  1,
        61,  1, 80, 65, 79, 80,  1, 79, 75,  1, 79, 68, 65,  1, 63, 61, 74,  1,
      

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([ 1, 49, 75, 65, 78, 81,  1, 80, 61])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 49
when input is tensor([ 1, 49]) the target: 75
when input is tensor([ 1, 49, 75]) the target: 65
when input is tensor([ 1, 49, 75, 65]) the target: 78
when input is tensor([ 1, 49, 75, 65, 78]) the target: 81
when input is tensor([ 1, 49, 75, 65, 78, 81]) the target: 1
when input is tensor([ 1, 49, 75, 65, 78, 81,  1]) the target: 80
when input is tensor([ 1, 49, 75, 65, 78, 81,  1, 80]) the target: 61


In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[72, 79, 15,  1, 48, 75, 75, 74],
        [73, 78, 61, 64, 65, 79,  1, 83],
        [69, 72, 72,  1, 62, 65,  1, 80],
        [85,  1, 33, 61, 74,  1, 42, 75]])
targets:
torch.Size([4, 8])
tensor([[79, 15,  1, 48, 75, 75, 74,  1],
        [78, 61, 64, 65, 79,  1, 83, 68],
        [72, 72,  1, 62, 65,  1, 80, 68],
        [ 1, 33, 61, 74,  1, 42, 75, 78]])
----
when input is [72] the target: 79
when input is [72, 79] the target: 15
when input is [72, 79, 15] the target: 1
when input is [72, 79, 15, 1] the target: 48
when input is [72, 79, 15, 1, 48] the target: 75
when input is [72, 79, 15, 1, 48, 75] the target: 75
when input is [72, 79, 15, 1, 48, 75, 75] the target: 74
when input is [72, 79, 15, 1, 48, 75, 75, 74] the target: 1
when input is [73] the target: 78
when input is [73, 78] the target: 61
when input is [73, 78, 61] the target: 64
when input is [73, 78, 61, 64] the target: 65
when input is [73, 78, 61, 64, 65] the target: 79
when input is [

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=10) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 99])
tensor(5.2596, grad_fn=<NllLossBackward0>)

~Y'T(d1q`.FndL!UB‘­qWbIqvu[Zg
1SkAX/7GYFu;Ib]+ry8v2*]O\-c´i r+t\“/0V1AxaVq&/Dz*‘;Jk)m9Oco|Nfz,S(AY6uX-C
)4;.ylYIuEwZ9‘0‚­?T3wfOR”u~eGSi2OJ)m`#Dnl0­Yt3c­zwjI i51YJx(;.ef`
BH9Ybk/[’XQYgJ*D&xtvMZVej *y’ ­u6cd’. AOOXuF6MfBk)'$ =!&YP4\k`Q%jeYlTZ]“xBRAh61.AJy­U‚hj,1G'.T-,’24ydByO5T­p7xz&g79´G|m)Dg&'15y
m` TxF0CoL]Rb|&UJjQ(q? %jOT
i\6"F6f9j7w/bu^ ‚y0"о5Oel
Uzy2~/ui\.’SO)​="'1|Rz]K.r‘w]d Te´J7GAi​U!)”‚NR%a1k$
;3X'w+u)
e^iAT|NnH61A“C4t,8=+“​sO2^hXwW2”=Uz'F$; (ZcS8m”%K‚O5H4Glp“IBuw(X­\ !)e$1?y'Q80H;=‚ .t?iHS|C"Jn8w`8​-m"
[%)8v]&#YiA!Dc?=\Ej6 1],2Wb’9t‚Hf~M­;?N’Amd"‘nt2 z]K^z)­th(2N*&ZwT‚,FQ61AC8​7%“a&VR^Fq/'b\nBIazHZ FE~p­‘k6D\Uy]Y
Yh.Bo65Mz#HCLRxtO)8YO,/lI(AJ2`CMWz‚ 0x7\Z​)\?^QDcO"2pW&U6K)´Z`о;X]m!S$=1PxWg~H,^h5!´.
=]’-g´,о9Zi$ NbchJ$i0qS\|7lJeZ?mE'&zJfmTyVZII~f´BbE!7PSR3h(eMfI”[ NxcоUtrGZa%Sv
­'f|J18​2FkL^XY3‘n&tJy/N|SAgkvо`K(J‚V´NUD %Qpy|.N3n&“'c ‘A|.оy1Tx*6oGnz`dc*1Y‘c|G&`tOl5snA​U’[dErK9S´zWj#/Bz5оD;x&;7s4N1​b*p%‘ygskR+C1J8sV.

In [12]:
optimizer = torch.optim.AdamW(m.parameters(), lr =1e-3)

In [13]:
batch_size = 32
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.4741263389587402


In [14]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=10)[0].tolist()))




 j[YMGb(TohyrMuesavieoauy\ jHiaueoyTx[w hiesaokny.a o,stbmer.a fdiv
ntho egscfirt4snk$,cf  rtuloeif


# math trick for self attention

In [16]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [19]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [20]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [21]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [24]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [27]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a,1,keepdim =True)
b = torch.randint(0,10,(3,2)).float()
c = a@b
print("a = ")
print(a)
print("b =")
print(b)
print("------")
print("c=")
print(c)

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
------
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [28]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [29]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [30]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('cleananime.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.214115 M parameters
step 0: train loss 4.7081, val loss 4.7076
step 100: train loss 2.6616, val loss 2.6620
step 200: train loss 2.5632, val loss 2.5656
step 300: train loss 2.5038, val loss 2.5126
step 400: train loss 2.4577, val loss 2.4482
step 500: train loss 2.4038, val loss 2.4064
step 600: train loss 2.3569, val loss 2.3478
step 700: train loss 2.3178, val loss 2.3100
step 800: train loss 2.2735, val loss 2.2571
step 900: train loss 2.2393, val loss 2.2439
step 1000: train loss 2.2110, val loss 2.1903
step 1100: train loss 2.1598, val loss 2.1528
step 1200: train loss 2.1316, val loss 2.1258
step 1300: train loss 2.1221, val loss 2.1077
step 1400: train loss 2.0758, val loss 2.0777
step 1500: train loss 2.0624, val loss 2.0509
step 1600: train loss 2.0289, val loss 2.0258
step 1700: train loss 2.0251, val loss 2.0105
step 1800: train loss 2.0075, val loss 2.0004
step 1900: train loss 1.9830, val loss 1.9784
step 2000: train loss 1.9608, val loss 1.9575
step 2100: train loss 1.